<a href="https://colab.research.google.com/github/diefgallegorio/Entrega-MLP-FIFA/blob/master/FIFA_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MLP FIFA

Installo tensorflow y keras

In [1]:
!pip install tensorflow==2.0.0-alpha0
# !pip install --upgrade tensorflow
# !pip install -U tensorflow-gpu==2.0.0 grpcio
!pip install --upgrade keras

Requirement already up-to-date: keras in /usr/local/lib/python3.6/dist-packages (2.3.1)


## Importo desde Drive la BD y el archivo de funciones para el preprocesamiento

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

from sklearn.impute import SimpleImputer 
from sklearn.model_selection import train_test_split, GridSearchCV,cross_val_score, cross_val_predict
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline

import tensorflow as tf
from tensorflow import keras

# para acceder a archivos desde google drive en google colaboratory

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#Importo base de datos housing

fifa = '1iyO_wm5Ws6ZT_VgMLQWUSosrk_Pydnxl'
downloaded = drive.CreateFile({'id': fifa})
downloaded.GetContentFile('fifa.csv') 

#Importo archivo de funciones para housing

Funciones = '1HNYPENq7gsP9KqhGevxCdCe87Z4e6YYZ'
downloaded = drive.CreateFile({'id': Funciones})
downloaded.GetContentFile('funciones_fifa.py') 

#Importo la función declarada en el archivo externo

from funciones_fifa import dummy_fifa, pre_exploratorio, code_euro

#Con pandas leo la base de datos y pruebo que cargue adecuadamente
Xdata = pd.read_csv('fifa.csv')

#Guardo columna de photo
pivote = Xdata['Photo']

#Columnas que se pueden borrar
col_drop = ['Unnamed: 0', 'Name','ID','Flag',
           'Club Logo','Loaned From']
Xdata.drop(columns = col_drop, inplace = True)

## Separo y codifico las imagenes en una columna a parte
En caso de que la url no sea válida, se pondrá 0 en el atributo, para posteriormente buscar todas las tuplas que tengan 0 en ese atributo para eliminar la tupla completa

In [0]:
def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

In [3]:
!pip install opencv-python

In [4]:
Xdata = Xdata.iloc[:500,:]
pivote = pivote.iloc[:500]
pivote

0      https://cdn.sofifa.org/players/4/19/158023.png
1       https://cdn.sofifa.org/players/4/19/20801.png
2      https://cdn.sofifa.org/players/4/19/190871.png
3      https://cdn.sofifa.org/players/4/19/193080.png
4      https://cdn.sofifa.org/players/4/19/192985.png
                            ...                      
495    https://cdn.sofifa.org/players/4/19/190778.png
496    https://cdn.sofifa.org/players/4/19/190544.png
497    https://cdn.sofifa.org/players/4/19/190286.png
498    https://cdn.sofifa.org/players/4/19/190264.png
499    https://cdn.sofifa.org/players/4/19/189805.png
Name: Photo, Length: 500, dtype: object

In [0]:
import cv2
import requests

for i in range(len(pivote)):
  try:
    url = pivote[i]
    resp = requests.get(url, stream=True).raw
    image = np.asarray(bytearray(resp.read()), dtype="uint8")
    image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    pivote[i] = image
  except:
    pivote[i] = np.zeros((48, 48, 3))

In [6]:
Xdata['Photo'] = pivote
Xdata['Photo'][1].shape

(48, 48, 3)

In [7]:
# response = requests.get(pivote[0]) 
# img = Image.open(BytesIO(response.content))
Xdata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 83 columns):
Age                         500 non-null int64
Photo                       491 non-null object
Nationality                 500 non-null object
Overall                     500 non-null int64
Potential                   500 non-null int64
Club                        499 non-null object
Value                       500 non-null object
Wage                        500 non-null object
Special                     500 non-null int64
Preferred Foot              500 non-null object
International Reputation    500 non-null float64
Weak Foot                   500 non-null float64
Skill Moves                 500 non-null float64
Work Rate                   500 non-null object
Body Type                   500 non-null object
Real Face                   500 non-null object
Position                    500 non-null object
Jersey Number               500 non-null float64
Joined                      485

## Divido el conjunto de datos, en: Xtest, Xtrain(Xtrain, Xvalid)

In [0]:
Xtest, Xtrain = train_test_split(Xdata,test_size=0.7)
Xvalid, Xtrain = train_test_split(Xtrain,test_size=0.7)

## Aplico pre-procesamiento

In [0]:
col_sal = "Release Clause"

ytrain = Xtrain[col_sal]
ytest = Xtest[col_sal]
yvalid = Xvalid[col_sal]
Xtrain.drop(columns=col_sal,inplace=True)
Xtest.drop(columns=col_sal,inplace=True)
Xvalid.drop(columns=col_sal,inplace=True)

ytrain = SimpleImputer(strategy="most_frequent").fit_transform(pd.DataFrame(ytrain)).reshape(-1)
ytrain = code_euro(ytrain)

ytest = SimpleImputer(strategy="most_frequent").fit_transform(pd.DataFrame(ytest)).reshape(-1)
ytest = code_euro(ytest)

In [0]:
col_euro = ['Value','Wage'];
col_stats = ['LS','ST','RS','LW','LF','CF','RF','RW','LAM',
             'CAM','RAM','LM','LCM','CM','RCM','RM','LWB',
             'LDM','CDM','RDM','RWB','LB','LCB','CB','RCB','RB'];

cat = ['Nationality','Club','Body Type','Position','Preferred Foot',
        'Work Rate','Real Face','Contract Valid Until']
items = [] 
for i in cat:
    items += [list(Xdata[i].value_counts().index)]
cat_usr = dict(zip(cat, items))

dummy = dummy_fifa(col_euro=col_euro,col_stats=col_stats,cat_usr = cat_usr)

In [0]:
#Columnas que se pueden borrar
pivote = Xtrain['Photo']
Xtrain.drop(columns = ['Photo'], inplace = True)
pivoteval = Xvalid['Photo']
Xvalid.drop(columns = ['Photo'], inplace = True)

Xtrain_pre = dummy.fit_transform(Xtrain)

In [13]:
Xtrain_pre.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 245 entries, 305 to 167
Data columns (total 81 columns):
Age                         245 non-null int64
Nationality                 245 non-null float64
Overall                     245 non-null int64
Potential                   245 non-null int64
Club                        245 non-null float64
Value                       245 non-null float64
Wage                        245 non-null float64
Special                     245 non-null int64
Preferred Foot              245 non-null float64
International Reputation    245 non-null float64
Weak Foot                   245 non-null float64
Skill Moves                 245 non-null float64
Work Rate                   245 non-null float64
Body Type                   245 non-null float64
Real Face                   245 non-null float64
Position                    245 non-null float64
Jersey Number               245 non-null float64
Joined                      245 non-null float64
Contract Valid Unti

In [25]:
Xtrain_pre.shape[1:]

(81,)

## Creo el modelo de red neuronal

Se tendrá 2 entradas:
1. Será el dataset de entrenamiento con 81  atributos, sin incluir fotos
2. Columna de fotos, con sus dimensiones correspondientes (48 x 48) en RGB

Cada entrada tendrá una camino de red diferente, pero se concatenan en un punto con el objetivo de tener en cuenta variables de las dos redes para tener 2 salidas: Clasificador del jugador (Bueno, regular o malo) y precio del jugador



In [0]:
from functools import partial

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")

#####111111#####

input_A = keras.layers.Input(shape = Xtrain_pre.shape[1:])

hidden1A = keras.layers.Dense(30, activation = 'relu')(input_A)
hidden2A = keras.layers.Dense(10, activation = 'tanh')(hidden1A)


#####222222#####
input_B = keras.layers.Input(shape=(48,48,3))
cnn1 = keras.layers.Conv2D(filters=64,input_shape=[48,48,3],kernel_size=7, activation='relu', padding="SAME")(input_B)
#input_B = DefaultConv2D(filters=64, kernel_size=7, input_shape=[48, 48, 3])
hidden1B = keras.layers.MaxPooling2D(pool_size=2)(cnn1)
hidden2B = DefaultConv2D(filters=128)(hidden1B)
hidden3B = keras.layers.MaxPooling2D(pool_size=2)(hidden2B)
hidden4B = DefaultConv2D(filters=256)(hidden3B)
hidden5B = keras.layers.Flatten()(hidden4B)
hidden6B = keras.layers.Dense(units=128, activation='relu')(hidden5B)

####CONCATENAR#####

concat = keras.layers.concatenate([hidden2A,hidden6B])

hidden1AB = keras.layers.Dense(10, activation = 'relu')(hidden1A)

######OUTPUT######
outputA = keras.layers.Dense(3, activation='softmax')(hidden1AB)
outputB = keras.layers.Dense(1)(hidden1AB)

model_2out = keras.Model(inputs=[input_A, input_B],outputs=[outputA,outputB])


In [18]:
model_2out.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 81)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 30)           2460        input_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 10)           310         dense[0][0]                      
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 3)            33          dense_3[0][0]                    
______________________________________________________________________________________________

In [19]:
hidden1 = model_2out.layers[1]
weights, biases = hidden1.get_weights()
weights

array([[ 0.0890159 , -0.04420316, -0.01709238, ..., -0.08295827,
         0.10282069, -0.02713013],
       [ 0.0662787 , -0.19433777, -0.11153138, ..., -0.20171705,
        -0.07712425,  0.22126424],
       [-0.11309493,  0.1704314 ,  0.2031435 , ..., -0.06899182,
         0.04266834,  0.05965728],
       ...,
       [ 0.04419878, -0.19053996,  0.12731183, ...,  0.06919277,
         0.12042743,  0.13327295],
       [-0.17686105,  0.06987873, -0.05722117, ...,  0.11341029,
         0.01964214, -0.22582841],
       [ 0.21716884,  0.17769733,  0.22790569, ..., -0.07516648,
        -0.12212407, -0.21957399]], dtype=float32)

In [20]:
print(biases.shape)
biases

(30,)


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [0]:
model_2out.compile(loss=keras.losses.mse,
              optimizer="sgd",
              metrics=[keras.losses.mae])

In [24]:
history = model_2out.fit(Xtrain, ytrain, epochs=10, validation_data=(Xvalid, yvalid))

ValueError: ignored